Simulation cell parameters

In [9]:
Nx = 100
Ny = 100
NxNy = Nx*Ny

dx = 0.5
dy = 0.5

0.5

Time integration parameters

In [10]:
nstep = 5000
nprint = 50
dtime = 1.e-4


0.0001

Material specific parameters

In [11]:
npart = 2
coefm = 5.0
coefk = 2.0
coefl = 5.0

dvol = 0.040
dvap = 0.002
dsur = 16.0
dgrb = 1.6

1.6

In [15]:
function micro_sint_pre(Nx,Ny, npart, iflag)
    if (iflag == 1)
        con = zeros(Nx,Ny)
        etas = zeros(Nx,Ny,npart)
        for ipart = 1:npart
            for i = 1 : Nx
                for j = 1 : Ny
                    con[i,j] = 0.0
                    etas[i,j,ipart] = 0.0
                end
            end
        end
    end
    if (iflag == 2)
        NxNy = Nx*Ny
        con = zeros(NxNy,1)
        etas = zeros(NxNy,npart)
    end
    
    # set particles
    if (npart != 2)
        R = 10.0
        
        xc[1] = 29.0
        yc[1] = 50.0
        
        xc[2] = 50.0
        yc[2] = 50.0
        
        xc[3] = 71.0
        yc[3] = 50.0
        
        xc[4] = 50.0
        yc[4] = 29.0
        
        xc[5] = 50.0
        yc[5] = 71.0
        
        xc[6] = 39.0
        yc[6] = 39.0
        
        xc[7] = 61.0
        yc[7] = 39.0
        
        xc[8] = 39.0
        yc[8] = 61.0
        
        xc[9] = 61.0
        yc[9] = 61.0
        
        for ipart = 1:npart
            Rx = R
            
            if (ipart > 5)
                Rx = 0.5 * R
            end
            
            for i = 1:Nx
                for j = 1:Ny
                    ii = (i-1)*Nx+j
                    
                    r1 = sqrt((i-xc[ipart])^2 + (j-yc[ipart])^2)
                    
                    if (r1 <= Rx)
                        if (iflag == 2)
                            con[ii] = 0.999
                            etas[ii,ipart] = 0.999
                        else
                            con[i,j] = 0.999
                            etas[i,j,ipart] = 0.999
                        end
                    end
                    
                end
            end
        end
    end
            
    if(npart == 2)
        R1 = 20.0
        R2 = 0.5*R1
                
        x1 = Nx/2
        y1 = 40.0
        y2 = 70.0
                
        for i = 1 : Nx
            for j = 1 : Ny
                ii = (i-1)*Nx + j
                        
                r1 = sqrt((i-x1)^2 + (j-y1)^2)
                r2 = sqrt((i-x1)^2 + (j-y2)^2)
                        
                if(r1 <= R1)
                    if (iflag == 2)
                        con[ii] = 0.999
                        etas[ii,1] = 0.999
                    else
                        con[i,j] = 0.999
                        etas[i,j,1] = 0.999
                    end
                end
                        
                if(r2 <= R2)
                    if (iflag == 2)
                        con[ii] = 0.999
                        etas[ii,1] = 0.0
                        etas[ii,2] = 0.999
                    else
                        con[i,j] = 0.999
                        etas[i,j,1] = 0.0
                        etas[i,j,2] = 0.999
                    end
                end
                        
            end
        end
    end 
    return npart, etas, con
end

micro_sint_pre (generic function with 1 method)

In [33]:
function free_energ_sint_v1(i,j,con,eta,etas,npart,iflag)
    A = 16.0
    B = 1.0
    
    dfdcon = 0.0
    dfdeta = 0.0
    
    if (iflag == 1)
        # derivative of free energy for concentration
        sum2 = 0.0
        sum3 = 0.0
        
        for ipart = 1: npart
            sum2 = sum2 + etas[i,j,ipart].^2
            sum3 = sum3 + etas[i,j,ipart].^3
        end
        dfdcon = B*(2.0*con[i,j] + 4.0*sum3 - 6.0*sum2) - 2.0*A*con[i,j]^2 .*
                (1.0 - con[i,j]) + 2.0*A*con[i,j]*(1.0-con[i,j])^2
    end
    if(iflag == 2)
        # derivative of free energy
        # for etas
        sum2 = 0.0
        for ipart = 1:npart
            sum2 = sum2 + etas[i,j,ipart]^2
        end
        dfdeta = B*(-12.0 * eta[i,j]^2 * (2.0 - con[i,j]) + 12.0 * eta[i,j] *
                   (1.0 - con[i,j]) + 12.0 * eta[i,j] * sum2)
    end
    return dfdcon, dfdeta
end

free_energ_sint_v1 (generic function with 1 method)

This function initialize the concentration and the order parameters in the 
simulation cell and defines the particle position either two particles or
nine particle simulations.

Prepare microstructure

In [25]:
iflag = 1
npart, etas, con = micro_sint_pre(Nx, Ny, npart, iflag)

eta = zeros(Nx,Ny)
# initialize eta
for i = 1 : Nx
    for j = 1 : Ny
        eta[i,j] = 0.0
    end
end


In [36]:
lap_con = zeros(Nx,Ny)
lap_eta = zeros(Nx,Ny)
dummy = zeros(Nx,Ny)
lap_dummy = zeros(Nx,Ny)

100×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [39]:
using Printf
using NBInclude
@nbinclude("write_vtk_file.ipynb")
for istep = 1:nstep
    # evolve concentration
    iflag = 1
    for i = 1 : Nx
        for j = 1 : Ny
            jp = j + 1
            jm = j - 1
            
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == (Nx+1))
                ip = 1
            end
            
            if (jm == 0)
                jm = Ny
            end
            if (jp == (Ny+1))
                jp = 1
            end
            
            hne = con[ip, j]
            hnw = con[im, j]
            hns = con[i, jm]
            hnn = con[i, jp]
            hnc = con[i,j]
            
            lap_con[i,j] = (hnw + hne + hns + hnn - 4.0*hnc)/(dx*dy)
            
          
            # derivative of free energy
            dfdcon, dfdeta = free_energ_sint_v1(i,j, con, eta, etas, npart, iflag)
           
            dummy[i,j] = dfdcon - 0.5*coefm*lap_con[i,j]
        end
    end
    
    for i = 1:Nx
        for j = 1:Ny
            jp = j + 1
            jm = j - 1
            
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == (Nx+1))
                ip = 1
            end
            
            if (jm == 0)
                jm = Ny
            end
            if (jp == (Ny+1))
                jp = 1
            end
            
            hne = dummy[ip,j]
            hnw = dummy[im,j]
            hnn = dummy[i,jp]
            hns = dummy[i,jm]
            hnc = dummy[i,j]
            
            lap_dummy[i,j] = (hnw + hne + hns + hnn - 4.0*hnc)/(dx*dy)
            
            # Mobility
            phi = con[i,j]^3 * (10.0 - 15.0*con[i,j] + 6.0 * con[i,j]^2)
            
            sum = 0.0
            for ipart = 1 : npart
                for jpart = 1 : npart
                    if (ipart != jpart)
                        sum = sum + etas[i,j,ipart] * etas[i,j,jpart]
                    end
                end
            end
            
            mobil = dvol * phi + dvap * (1.0-phi) + dsur * con[i,j]*(1.0 - con[i,j]) +
                    2.0 * dgrb * sum
            
            # time integration
            con[i,j] = con[i,j] + dtime * mobil * lap_dummy[i,j]
            
            # for small deviations
            if (con[i,j] >= 0.9999)
                con[i,j] = 0.9999
            end
            
            if (con[i,j] < 0.00001)
                con[i,j] = 0.00001
            end
            
        end
    end
    
    # evolve etas:
    
    iflag = 2
    
    for ipart = 1 : npart
        for i = 1 : Nx
            for j = 1 : Ny
                eta[i,j] = etas[i,j,ipart]
            end
        end
        
        for i = 1:Nx
            for j = 1:Ny
                jp = j + 1
                jm = j - 1
            
                ip = i + 1
                im = i - 1
            
                if (im == 0)
                    im = Nx
                end
                if (ip == (Nx+1))
                    ip = 1
                end
            
                if (jm == 0)
                    jm = Ny
                end
                if (jp == (Ny+1))
                    jp = 1
                end
        
                hne = eta[ip,j]
                hnw = eta[im,j]
                hnn = eta[i,jp]
                hns = eta[i,jm]
                hnc = eta[i,j]
                
                lap_eta[i,j] = (hnw + hne + hns + hnn - 4.0*hnc)/(dx*dy)
                
                # derivative of free energy
                dfdcon, dfdeta = free_energ_sint_v1(i,j,con,eta,etas,npart,iflag)
                
                # Time integration
                eta[i,j] = eta[i,j] - dtime * coefl * (dfdeta - 0.5*coefk*lap_eta[i,j])
                
                # For small deviations
                if (eta[i,j] >= 0.99999)
                    eta[i,j] = 0.99999
                end
                
                if (eta[i,j] < 0.00001)
                    eta[i,j] = 0.00001
                end
                
            end
        end
        
        for i = 1 : Nx
            for j = 1 : Ny
                etas[i,j,ipart] = eta[i,j]
            end
        end
        
    end
    
    # print results
    if ((mod(istep,nprint)== 0) || (istep == 1))
        @printf("done step: %5d\n", istep)
        
        # write vtk file
        phi2 = zeros(Nx, Ny)
        for i = 1:Nx
            for j = 1:Ny
                for ipart = 1:npart
                    phi2[i,j] = phi2[i,j] + etas[i,j,ipart]^2
                end
            end
        end
        # write data to the vtk files
        write_vtk_grid_values(Nx, Ny, dx, dy, istep, con)
    end
end

        
            

┌ Info: Precompiling NBInclude [0db19996-df87-5ea3-a455-e3a50d440464]
└ @ Base loading.jl:1278


done step:     1
done step:    50
done step:   100
done step:   150
done step:   200
done step:   250
done step:   300
done step:   350
done step:   400
done step:   450
done step:   500
done step:   550
done step:   600
done step:   650
done step:   700
done step:   750
done step:   800
done step:   850
done step:   900
done step:   950
done step:  1000
done step:  1050
done step:  1100
done step:  1150
done step:  1200
done step:  1250
done step:  1300
done step:  1350
done step:  1400
done step:  1450
done step:  1500
done step:  1550
done step:  1600
done step:  1650
done step:  1700
done step:  1750
done step:  1800
done step:  1850
done step:  1900
done step:  1950
done step:  2000
done step:  2050
done step:  2100
done step:  2150
done step:  2200
done step:  2250
done step:  2300
done step:  2350
done step:  2400
done step:  2450
done step:  2500
done step:  2550
done step:  2600
done step:  2650
done step:  2700
done step:  2750
done step:  2800
done step:  2850
done step:  29